In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# === CONFIGURATION ===
IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 25
FOLDS = 10
DATASET_DIR = "augmented_dataset"  # üëà Folder containing class-named subfolders

# === STEP 1: Load all image paths and labels from folder structure ===
data = []
for class_name in os.listdir(DATASET_DIR):
    class_path = os.path.join(DATASET_DIR, class_name)
    if os.path.isdir(class_path):
        for img in os.listdir(class_path):
            if img.lower().endswith(('.jpg', '.jpeg', '.png')):
                data.append([os.path.join(class_path, img), class_name])

df = pd.DataFrame(data, columns=["filename", "class"])
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# === STEP 2: Label Encoding ===
label_map = {label: idx for idx, label in enumerate(sorted(df["class"].unique()))}
df["class_idx"] = df["class"].map(label_map)
NUM_CLASSES = len(label_map)

# === STEP 3: CNN Model ===
def build_cnn_model(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), num_classes=NUM_CLASSES):
    model = models.Sequential([
        layers.Input(shape=input_shape),

        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# === STEP 4: Data Generator ===
datagen = ImageDataGenerator(rescale=1./255)

# === STEP 5: K-Fold Cross-Validation ===
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)
fold_accuracies = []
fold_histories = []

for fold, (train_idx, val_idx) in enumerate(skf.split(df["filename"], df["class_idx"])):
    print(f"\nüü¶ Fold {fold+1}/{FOLDS}")

    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    train_gen = datagen.flow_from_dataframe(
        train_df,
        x_col="filename",
        y_col="class",
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        class_mode="categorical",
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42
    )

    val_gen = datagen.flow_from_dataframe(
        val_df,
        x_col="filename",
        y_col="class",
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        class_mode="categorical",
        batch_size=BATCH_SIZE,
        shuffle=False
    )

    model = build_cnn_model()
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=EPOCHS,
        callbacks=[early_stop, lr_reduce],
        verbose=1
    )

    val_loss, val_acc = model.evaluate(val_gen, verbose=0)
    fold_histories.append(history.history)
    fold_accuracies.append(val_acc)
    print(f"‚úÖ Fold {fold+1} Accuracy: {val_acc:.4f}")

# === STEP 6: Summary ===
mean_acc = np.mean(fold_accuracies)
std_acc = np.std(fold_accuracies)
print(f"\nüìä 5-Fold Validation Accuracy: {mean_acc:.4f} ¬± {std_acc:.4f}")

# === STEP 7: Accuracy/Loss Plot ===
def plot_kfold_results(histories):
    fig, axs = plt.subplots(2, 1, figsize=(12, 10))

    for i, history in enumerate(histories):
        axs[0].plot(history['accuracy'], label=f'Train Fold {i+1}')
        axs[0].plot(history['val_accuracy'], linestyle='--', label=f'Val Fold {i+1}')
        axs[1].plot(history['loss'], label=f'Train Fold {i+1}')
        axs[1].plot(history['val_loss'], linestyle='--', label=f'Val Fold {i+1}')

    axs[0].set_title('Training vs Validation Accuracy (All Folds)')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend()
    axs[0].grid()

    axs[1].set_title('Training vs Validation Loss (All Folds)')
    axs[1].set_ylabel('Loss')
    axs[1].legend()
    axs[1].grid()

    plt.tight_layout()
    plt.show()

plot_kfold_results(fold_histories)


üü¶ Fold 1/10
Found 26727 validated image filenames belonging to 10 classes.
Found 2970 validated image filenames belonging to 10 classes.


2025-06-30 17:53:31.153539: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-06-30 17:53:31.153580: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-06-30 17:53:31.153584: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2025-06-30 17:53:31.153618: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-30 17:53:31.153629: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/simarkalsi/Projects/leaf K-cross/kcross/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should ca

Epoch 1/25


2025-06-30 17:53:31.928105: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 82s 96ms/step - accuracy: 0.4005 - loss: 1.7492 - val_accuracy: 0.7313 - val_loss: 0.8361 - learning_rate: 1.0000e-04
Epoch 2/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 78s 94ms/step - accuracy: 0.6720 - loss: 0.9416 - val_accuracy: 0.7896 - val_loss: 0.6603 - learning_rate: 1.0000e-04
Epoch 3/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 96ms/step - accuracy: 0.7356 - loss: 0.7704 - val_accuracy: 0.7835 - val_loss: 0.6634 - learning_rate: 1.0000e-04
Epoch 4/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 84s 100ms/step - accuracy: 0.7819 - loss: 0.6401 - val_accuracy: 0.7929 - val_loss: 0.6052 - learning_rate: 1.0000e-04
Epoch 5/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 79s 95ms/step - accuracy: 0.8072 - loss: 0.5717 - val_accuracy: 0.8347 - val_loss: 0.4709 - learning_rate: 1.0000e-04
Epoch 6/25
836/836 ‚

...

836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 83s 97ms/step - accuracy: 0.3680 - loss: 1.8262 - val_accuracy: 0.6676 - val_loss: 0.8943 - learning_rate: 1.0000e-04
Epoch 2/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 96ms/step - accuracy: 0.6316 - loss: 1.0408 - val_accuracy: 0.6942 - val_loss: 0.8854 - learning_rate: 1.0000e-04
Epoch 3/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 96ms/step - accuracy: 0.7123 - loss: 0.8350 - val_accuracy: 0.7649 - val_loss: 0.6574 - learning_rate: 1.0000e-04
Epoch 4/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 96ms/step - accuracy: 0.7592 - loss: 0.7048 - val_accuracy: 0.8302 - val_loss: 0.4896 - learning_rate: 1.0000e-04
Epoch 5/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 96ms/step - accuracy: 0.7887 - loss: 0.6271 - val_accuracy: 0.8936 - val_loss: 0.3544 - learning_rate: 1.0000e-04
Epoch 6/25
836/836 ‚î

/Users/simarkalsi/Projects/leaf K-cross/kcross/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 82s 96ms/step - accuracy: 0.4040 - loss: 1.7340 - val_accuracy: 0.6911 - val_loss: 0.9150 - learning_rate: 1.0000e-04
Epoch 2/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 95ms/step - accuracy: 0.6564 - loss: 0.9889 - val_accuracy: 0.6868 - val_loss: 0.9029 - learning_rate: 1.0000e-04
Epoch 3/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 95ms/step - accuracy: 0.7321 - loss: 0.7839 - val_accuracy: 0.8151 - val_loss: 0.5171 - learning_rate: 1.0000e-04
Epoch 4/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 95ms/step - accuracy: 0.7754 - loss: 0.6679 - val_accuracy: 0.8289 - val_loss: 0.4914 - learning_rate: 1.0000e-04
Epoch 5/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 95ms/step - accuracy: 0.8093 - loss: 0.5698 - val_accuracy: 0.8666 - val_loss: 0.3915 - learning_rate: 1.0000e-04
Epoch 6/25
836/836 ‚î

/Users/simarkalsi/Projects/leaf K-cross/kcross/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 82s 96ms/step - accuracy: 0.3877 - loss: 1.7550 - val_accuracy: 0.6426 - val_loss: 1.0241 - learning_rate: 1.0000e-04
Epoch 2/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 96ms/step - accuracy: 0.6568 - loss: 0.9845 - val_accuracy: 0.7339 - val_loss: 0.8020 - learning_rate: 1.0000e-04
Epoch 3/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 80s 96ms/step - accuracy: 0.7441 - loss: 0.7623 - val_accuracy: 0.8077 - val_loss: 0.5682 - learning_rate: 1.0000e-04
Epoch 4/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 81s 97ms/step - accuracy: 0.7822 - loss: 0.6338 - val_accuracy: 0.8656 - val_loss: 0.4158 - learning_rate: 1.0000e-04
Epoch 5/25
836/836 ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 81s 96ms/step - accuracy: 0.8082 - loss: 0.5563 - val_accuracy: 0.8222 - val_loss: 0.5105 - learning_rate: 1.0000e-04
Epoch 6/25
836/836 ‚î

In [2]:
for i, acc in enumerate(fold_accuracies):
    print(f"Fold {i+1} Accuracy: {acc:.4f}")
print(f"\nüìä Final K-Fold Avg Accuracy: {mean_acc:.4f} ¬± {std_acc:.4f}")


Fold 1 Accuracy: 0.9633
Fold 2 Accuracy: 0.9667
Fold 3 Accuracy: 0.9545
Fold 4 Accuracy: 0.9670
Fold 5 Accuracy: 0.9492
Fold 6 Accuracy: 0.9603
Fold 7 Accuracy: 0.9606
Fold 8 Accuracy: 0.9643
Fold 9 Accuracy: 0.9589
Fold 10 Accuracy: 0.9673

üìä Final K-Fold Avg Accuracy: 0.9612 ¬± 0.0056


In [3]:
model.save(f"cnn_kfold_model_fold{fold+1}.keras")

In [5]:
import matplotlib.pyplot as plt

def plot_cnn_fold_learning_curves(histories, model_name="CNN", fold_type="5-Fold"):
    """
    Plots training vs validation accuracy and loss across multiple folds
    to check for overfitting/underfitting.
    
    Parameters:
    - histories: List of history.history dictionaries (1 per fold)
    - model_name: Name to display in the title (e.g. 'CNN')
    - fold_type: '5-Fold' or '10-Fold'
    """
    fig, axs = plt.subplots(1, 2, figsize=(14, 5))

    # Accuracy plot
    for i, history in enumerate(histories):
        axs[0].plot(history['accuracy'], label=f'Train Fold {i+1}')
        axs[0].plot(history['val_accuracy'], linestyle='--', label=f'Val Fold {i+1}')

    axs[0].set_title(f'{model_name} {fold_type} Accuracy per Fold')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epochs')
    axs[0].legend(loc='lower right')
    axs[0].grid()

    # Loss plot
    for i, history in enumerate(histories):
        axs[1].plot(history['loss'], label=f'Train Fold {i+1}')
        axs[1].plot(history['val_loss'], linestyle='--', label=f'Val Fold {i+1}')

    axs[1].set_title(f'{model_name} {fold_type} Loss per Fold')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epochs')
    axs[1].legend(loc='upper right')
    axs[1].grid()

    plt.tight_layout()
    plt.show()


In [6]:
plot_cnn_fold_learning_curves(fold_histories, model_name="CNN", fold_type="5-Fold")
